In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Configuración inicial
np.random.seed(42)
turbinas = ['Turbina 1', 'Turbina 2', 'Turbina 3']
años = [2023, 2024]
meses = range(1, 13)

# Generar datos
datos = []
for año in años:
    for mes in meses:
        for turbina in turbinas:
            # Base + variabilidad estacional (más viento en invierno)
            base = 800 if mes in [11, 12, 1, 2] else 500
            generacion = base * np.random.uniform(0.85, 1.15)  # Variabilidad

            # Turbina 2 es más eficiente
            if turbina == 'Turbina 2':
                generacion *= 1.08

            datos.append({
                'Fecha': datetime(año, mes, 1),
                'Año': año,
                'Mes': mes,
                'Turbina': turbina,
                'Generacion_MWh': round(generacion, 2),
                'Horas_Operacion': np.random.randint(650, 730),  # 650-730 horas/mes
                'Velocidad_Viento_Promedio': round(np.random.uniform(6.5, 8.5), 1)
            })

df_generacion = pd.DataFrame(datos)
df_generacion.to_csv('datos_generacion_eolica.csv', index=False)

In [ ]:
# Curva de potencia teórica y real
velocidades = np.linspace(0, 25, 26)
curva_potencia = []
for v in velocidades:
    if v < 3:  # Cut-in speed
        potencia = 0
    elif 3 <= v < 12:  # Rango operativo
        potencia = 1500 * (v**3 / 12**3)  # Cúbica típica
    elif 12 <= v <= 25:  # Rated power
        potencia = 1500
    else:  # Cut-out speed
        potencia = 0

    # Añadir variabilidad real
    potencia_real = potencia * np.random.uniform(0.92, 1.05)

    curva_potencia.append({
        'Velocidad_Viento': v,
        'Potencia_Teorica_kW': round(potencia, 2),
        'Potencia_Real_kW': round(max(0, potencia_real), 2)
    })

df_curva_potencia = pd.DataFrame(curva_potencia)
df_curva_potencia.to_csv('curva_potencia_turbina.csv', index=False)

In [ ]:
direcciones = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
rosa_vientos = []

for direccion in direcciones:
    # Frecuencia relativa por dirección (ejemplo: más vientos del NE)
    if direccion == 'NE':
        freq = np.random.uniform(20, 25)
    elif direccion in ['N', 'E']:
        freq = np.random.uniform(12, 18)
    else:
        freq = np.random.uniform(5, 10)

    # Velocidad promedio por dirección
    velocidad = np.random.uniform(6, 9) if direccion == 'NE' else np.random.uniform(4, 7)

    rosa_vientos.append({
        'Direccion': direccion,
        'Frecuencia_Relativa': round(freq, 2),
        'Velocidad_Promedio_m/s': round(velocidad, 1)
    })

df_rosa_vientos = pd.DataFrame(rosa_vientos)
df_rosa_vientos.to_csv('rosa_vientos.csv', index=False)

In [ ]:
# Disponibilidad y fallos por turbina
disponibilidad = []
for turbina in turbinas:
    for mes in meses:
        horas_mes = 730  # Horas en un mes
        fallos = np.random.poisson(0.5)  # Número de fallos (distribución Poisson)
        horas_fallo = sum(np.random.exponential(8, fallos)) if fallos > 0 else 0

        disponibilidad.append({
            'Turbina': turbina,
            'Mes': mes,
            'Horas_Disponibles': round(horas_mes - horas_fallo, 2),
            'Horas_Fallo': round(horas_fallo, 2),
            'Numero_Fallos': fallos
        })

df_disponibilidad = pd.DataFrame(disponibilidad)
df_disponibilidad.to_csv('disponibilidad_turbinas.csv', index=False)

In [ ]:
# Eficiencia comparada entre turbinas
eficiencia = []
factores_eficiencia = {'Turbina 1': 1.0, 'Turbina 2': 1.08, 'Turbina 3': 0.97}

for turbina, factor in factores_eficiencia.items():
    for velocidad in range(4, 13):
        eficiencia_real = (0.3 + 0.6 * (velocidad-4)/(12-4)) * factor * np.random.uniform(0.95, 1.05)

        eficiencia.append({
            'Turbina': turbina,
            'Velocidad_Viento_m/s': velocidad,
            'Eficiencia': round(min(0.99, max(0.25, eficiencia_real)), 3)  # Entre 25% y 99%
        })

df_eficiencia = pd.DataFrame(eficiencia)
df_eficiencia.to_csv('eficiencia_turbinas.csv', index=False)

In [ ]:
# Script completo que genera todos los datasets
import os

# Crear directorio si no existe
os.makedirs('datos_eolicos', exist_ok=True)

# Generar todos los datasets
datasets = {
    'generacion': df_generacion,
    'curva_potencia': df_curva_potencia,
    'rosa_vientos': df_rosa_vientos,
    'disponibilidad': df_disponibilidad,
    'eficiencia': df_eficiencia
}

# Guardar todos los archivos
for nombre, df in datasets.items():
    df.to_csv(f'datos_eolicos/{nombre}.csv', index=False)

print("Todos los datasets han sido generados y guardados en la carpeta 'datos_eolicos'")

Todos los datasets han sido generados y guardados en la carpeta 'datos_eolicos'


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

# Configuración inicial
np.random.seed(42)
turbinas = ['Turbina 1', 'Turbina 2', 'Turbina 3']
años = [2022, 2023, 2024]
meses = range(1, 13)
direcciones_viento = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']

# =============================================
# 1. DATOS PARA EL RESUMEN
# =============================================

print("Generando datos para el Resumen...")

# 1.1 Generación Mensual por Turbina
datos_generacion = []
for año in años:
    for mes in meses:
        for turbina in turbinas:
            # Base + variabilidad estacional (más viento en invierno)
            base = 800 if mes in [11, 12, 1, 2] else 500
            generacion = base * np.random.uniform(0.85, 1.15)

            # Turbina 2 es más eficiente
            if turbina == 'Turbina 2':
                generacion *= 1.08
            elif turbina == 'Turbina 3':
                generacion *= 0.97

            datos_generacion.append({
                'Fecha': datetime(año, mes, 1).strftime('%Y-%m-%d'),
                'Año': año,
                'Mes': mes,
                'Mes_Texto': datetime(año, mes, 1).strftime('%b'),
                'Turbina': turbina,
                'Generacion_MWh': round(generacion, 2),
                'Horas_Operacion': np.random.randint(650, 730),
                'Velocidad_Viento_Promedio': round(np.random.uniform(6.5, 8.5), 1)
            })

df_generacion = pd.DataFrame(datos_generacion)

# 1.2 Factor de Capacidad Mensual
df_factor_capacidad = df_generacion.copy()
df_factor_capacidad['Factor_Capacidad'] = round(
    df_factor_capacidad['Generacion_MWh'] / (2000 * df_factor_capacidad['Horas_Operacion']/730) * 100, 2
)

# 1.3 Disponibilidad de Turbinas
datos_disponibilidad = []
for turbina in turbinas:
    for año in años:
        for mes in meses:
            horas_mes = 730
            fallos = np.random.poisson(0.7)
            horas_fallo = sum(np.random.exponential(10, fallos)) if fallos > 0 else 0

            datos_disponibilidad.append({
                'Turbina': turbina,
                'Año': año,
                'Mes': mes,
                'Horas_Disponibles': round(horas_mes - horas_fallo, 2),
                'Horas_Fallo': round(horas_fallo, 2),
                'Numero_Fallos': fallos,
                'Disponibilidad_Porcentaje': round((horas_mes - horas_fallo)/horas_mes * 100, 2)
            })

df_disponibilidad = pd.DataFrame(datos_disponibilidad)

# 1.4 Eficiencia Comparada
datos_eficiencia = []
factores_eficiencia = {'Turbina 1': 1.0, 'Turbina 2': 1.08, 'Turbina 3': 0.97}

for turbina, factor in factores_eficiencia.items():
    for velocidad in range(4, 25):
        eficiencia_real = (0.3 + 0.6 * (velocidad-4)/(24-4)) * factor * np.random.uniform(0.95, 1.05)

        datos_eficiencia.append({
            'Turbina': turbina,
            'Velocidad_Viento_m/s': velocidad,
            'Eficiencia': round(min(0.99, max(0.25, eficiencia_real)), 3)
        })

df_eficiencia = pd.DataFrame(datos_eficiencia)

# =============================================
# 2. DATOS PARA TENDENCIAS
# =============================================

print("Generando datos para Tendencias...")

# 2.1 Tendencia Anual de Generación
df_tendencia_anual = df_generacion.groupby(['Año', 'Turbina'])['Generacion_MWh'].sum().reset_index()
df_tendencia_anual['Generacion_GWh'] = round(df_tendencia_anual['Generacion_MWh'] / 1000, 2)

# 2.2 Comparación entre Turbinas
df_comparacion_turbinas = df_generacion.groupby(['Turbina', 'Mes'])['Generacion_MWh'].mean().reset_index()
df_comparacion_turbinas['Rendimiento_Relativo'] = round(
    df_comparacion_turbinas.groupby('Mes')['Generacion_MWh'].apply(
        lambda x: x / x.mean()
    ), 2
)

# =============================================
# 3. DATOS PARA VIENTO
# =============================================

print("Generando datos para Viento...")

# 3.1 Rosa de los Vientos
datos_rosa_vientos = []
for direccion in direcciones_viento:
    # Frecuencia relativa por dirección (ejemplo: más vientos del NE)
    if direccion == 'NE':
        freq = np.random.uniform(20, 25)
    elif direccion in ['N', 'E']:
        freq = np.random.uniform(12, 18)
    else:
        freq = np.random.uniform(5, 10)

    # Velocidad promedio por dirección
    velocidad = np.random.uniform(6, 9) if direccion == 'NE' else np.random.uniform(4, 7)

    datos_rosa_vientos.append({
        'Direccion': direccion,
        'Frecuencia_Relativa': round(freq, 2),
        'Velocidad_Promedio_m/s': round(velocidad, 1)
    })

df_rosa_vientos = pd.DataFrame(datos_rosa_vientos)

# 3.2 Curva de Potencia
velocidades = np.linspace(0, 25, 26)
datos_curva_potencia = []
for v in velocidades:
    if v < 3:  # Cut-in speed
        potencia = 0
    elif 3 <= v < 12:  # Rango operativo
        potencia = 1500 * (v**3 / 12**3)  # Cúbica típica
    elif 12 <= v <= 25:  # Rated power
        potencia = 1500
    else:  # Cut-out speed
        potencia = 0

    # Añadir variabilidad real
    potencia_real = potencia * np.random.uniform(0.92, 1.05)

    datos_curva_potencia.append({
        'Velocidad_Viento': v,
        'Potencia_Teorica_kW': round(potencia, 2),
        'Potencia_Real_kW': round(max(0, potencia_real), 2),
        'Eficiencia_Relativa': round(potencia_real/max(1, potencia), 2) if v >= 3 else 0
    })

df_curva_potencia = pd.DataFrame(datos_curva_potencia)

# =============================================
# GUARDAR TODOS LOS DATASETS
# =============================================

# Crear directorio si no existe
os.makedirs('datos_eolicos', exist_ok=True)

# Diccionario con todos los datasets
datasets = {
    # Resumen
    'generacion_mensual': df_generacion,
    'factor_capacidad': df_factor_capacidad,
    'disponibilidad_turbinas': df_disponibilidad,
    'eficiencia_comparada': df_eficiencia,

    # Tendencias
    'tendencia_anual': df_tendencia_anual,
    'comparacion_turbinas': df_comparacion_turbinas,

    # Viento
    'rosa_vientos': df_rosa_vientos,
    'curva_potencia': df_curva_potencia
}

# Guardar todos los archivos
for nombre, df in datasets.items():
    df.to_csv(f'datos_eolicos/{nombre}.csv', index=False)
    print(f"Dataset {nombre}.csv generado con {len(df)} registros")

print("\n¡Todos los datasets han sido generados exitosamente en la carpeta 'datos_eolicos'!")

Generando datos para el Resumen...
Generando datos para Tendencias...


TypeError: incompatible index of inserted column with frame index

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from datetime import datetime
import os

# Configurar paleta de colores
paleta_colores = ['#017c42', '#209559', '#3eae6f', '#5dc686', '#7bdf9c']
paleta_secundaria = ['#017c42', '#3eae6f', '#7bdf9c']  # 3 tonos para diferenciar turbinas

# Crear directorio para gráficos
os.makedirs('graficas_eolicas', exist_ok=True)

# =================================================================
# 1. GENERAR DATOS CON COLUMNAS NECESARIAS
# =================================================================

# Configuración inicial
np.random.seed(42)
turbinas = ['Turbina 1', 'Turbina 2', 'Turbina 3']
años = [2022, 2023, 2024]
meses = range(1, 13)
nombres_meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun',
                 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
direcciones_viento = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']

# 1.1 Generación Mensual por Turbina
datos_generacion = []
for año in años:
    for mes in meses:
        for turbina in turbinas:
            base = 800 if mes in [11, 12, 1, 2] else 500
            generacion = base * np.random.uniform(0.85, 1.15)

            if turbina == 'Turbina 2':
                generacion *= 1.08
            elif turbina == 'Turbina 3':
                generacion *= 0.97

            datos_generacion.append({
                'Fecha': datetime(año, mes, 1).strftime('%Y-%m-%d'),
                'Año': año,
                'Mes': mes,
                'Mes_Texto': nombres_meses[mes-1],
                'Turbina': turbina,
                'Generacion_MWh': round(generacion, 2),
                'Horas_Operacion': np.random.randint(650, 730),
                'Velocidad_Viento_Promedio': round(np.random.uniform(6.5, 8.5), 1)
            })

df_generacion = pd.DataFrame(datos_generacion)

# 1.2 Factor de Capacidad Mensual
df_factor_capacidad = df_generacion.copy()
df_factor_capacidad['Factor_Capacidad'] = round(
    df_factor_capacidad['Generacion_MWh'] / (2000 * df_factor_capacidad['Horas_Operacion']/730) * 100, 2
)

# 1.3 Disponibilidad de Turbinas (CORREGIDO: añadir Mes_Texto)
datos_disponibilidad = []
for turbina in turbinas:
    for año in años:
        for mes in meses:
            horas_mes = 730
            fallos = np.random.poisson(0.7)
            horas_fallo = sum(np.random.exponential(10, fallos)) if fallos > 0 else 0

            datos_disponibilidad.append({
                'Turbina': turbina,
                'Año': año,
                'Mes': mes,
                'Mes_Texto': nombres_meses[mes-1],  # Añadimos esta columna
                'Horas_Disponibles': round(horas_mes - horas_fallo, 2),
                'Horas_Fallo': round(horas_fallo, 2),
                'Numero_Fallos': fallos,
                'Disponibilidad_Porcentaje': round((horas_mes - horas_fallo)/horas_mes * 100, 2)
            })

df_disponibilidad = pd.DataFrame(datos_disponibilidad)

# 1.4 Eficiencia Comparada
datos_eficiencia = []
factores_eficiencia = {'Turbina 1': 1.0, 'Turbina 2': 1.08, 'Turbina 3': 0.97}

for turbina, factor in factores_eficiencia.items():
    for velocidad in range(4, 25):
        eficiencia_real = (0.3 + 0.6 * (velocidad-4)/(24-4)) * factor * np.random.uniform(0.95, 1.05)

        datos_eficiencia.append({
            'Turbina': turbina,
            'Velocidad_Viento_m/s': velocidad,
            'Eficiencia': round(min(0.99, max(0.25, eficiencia_real)), 3)
        })

df_eficiencia = pd.DataFrame(datos_eficiencia)

# 1.5 Datos para tendencias
df_tendencia_anual = df_generacion.groupby(['Año', 'Turbina'])['Generacion_MWh'].sum().reset_index()
df_tendencia_anual['Generacion_GWh'] = round(df_tendencia_anual['Generacion_MWh'] / 1000, 2)

# 1.6 Rosa de los Vientos
datos_rosa_vientos = []
for direccion in direcciones_viento:
    if direccion == 'NE':
        freq = np.random.uniform(20, 25)
    elif direccion in ['N', 'E']:
        freq = np.random.uniform(12, 18)
    else:
        freq = np.random.uniform(5, 10)

    velocidad = np.random.uniform(6, 9) if direccion == 'NE' else np.random.uniform(4, 7)

    datos_rosa_vientos.append({
        'Direccion': direccion,
        'Frecuencia_Relativa': round(freq, 2),
        'Velocidad_Promedio_m/s': round(velocidad, 1)
    })

df_rosa_vientos = pd.DataFrame(datos_rosa_vientos)

# 1.7 Curva de Potencia
velocidades = np.linspace(0, 25, 26)
datos_curva_potencia = []
for v in velocidades:
    if v < 3:
        potencia = 0
    elif 3 <= v < 12:
        potencia = 1500 * (v**3 / 12**3)
    elif 12 <= v <= 25:
        potencia = 1500
    else:
        potencia = 0

    potencia_real = potencia * np.random.uniform(0.92, 1.05)

    datos_curva_potencia.append({
        'Velocidad_Viento': v,
        'Potencia_Teorica_kW': round(potencia, 2),
        'Potencia_Real_kW': round(max(0, potencia_real), 2),
        'Eficiencia_Relativa': round(potencia_real/max(1, potencia), 2) if v >= 3 else 0
    })

df_curva_potencia = pd.DataFrame(datos_curva_potencia)

# =================================================================
# 2. GENERACIÓN DE GRÁFICOS CORREGIDOS
# =================================================================

def guardar_grafico(fig, nombre_archivo):
    """Guarda un gráfico Plotly como HTML"""
    ruta = f'graficas_eolicas/{nombre_archivo}.html'
    fig.write_html(ruta, full_html=False, include_plotlyjs='cdn')
    print(f'Gráfico guardado: {ruta}')

# 2.1 Generación Mensual por Turbina
fig_generacion = px.line(
    df_generacion,
    x='Mes_Texto',
    y='Generacion_MWh',
    color='Turbina',
    color_discrete_sequence=paleta_secundaria,
    facet_col='Año',
    title='Generación Mensual por Turbina (MWh)',
    labels={'Generacion_MWh': 'Generación (MWh)', 'Mes_Texto': 'Mes'},
    height=500
)
fig_generacion.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='#f8f9fa',
    hovermode='x unified'
)
guardar_grafico(fig_generacion, 'generacion_mensual')

# 2.2 Factor de Capacidad Mensual
fig_factor = px.area(
    df_factor_capacidad,
    x='Mes_Texto',
    y='Factor_Capacidad',
    color='Turbina',
    color_discrete_sequence=paleta_secundaria,
    facet_col='Año',
    title='Factor de Capacidad Mensual (%)',
    labels={'Factor_Capacidad': 'Factor de Capacidad (%)', 'Mes_Texto': 'Mes'},
    height=500
)
fig_factor.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='#f8f9fa',
    yaxis_range=[0, 100]
)
guardar_grafico(fig_factor, 'factor_capacidad')

# 2.3 Disponibilidad de Turbinas (CORREGIDO)
fig_disponibilidad = px.bar(
    df_disponibilidad,
    x='Mes_Texto',
    y='Disponibilidad_Porcentaje',
    color='Turbina',
    color_discrete_sequence=paleta_secundaria,
    barmode='group',
    facet_col='Año',
    title='Disponibilidad Mensual por Turbina (%)',
    labels={'Disponibilidad_Porcentaje': 'Disponibilidad (%)', 'Mes_Texto': 'Mes'},
    height=500
)
fig_disponibilidad.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='#f8f9fa',
    yaxis_range=[85, 100]
)
guardar_grafico(fig_disponibilidad, 'disponibilidad_turbinas')

# 2.4 Eficiencia Comparada
fig_eficiencia = px.line(
    df_eficiencia,
    x='Velocidad_Viento_m/s',
    y='Eficiencia',
    color='Turbina',
    color_discrete_sequence=paleta_secundaria,
    title='Eficiencia Comparada por Velocidad de Viento',
    labels={'Eficiencia': 'Eficiencia (%)', 'Velocidad_Viento_m/s': 'Velocidad del Viento (m/s)'},
    height=500
)
fig_eficiencia.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='#f8f9fa',
    yaxis_tickformat=',.0%',
    xaxis_range=[3, 25]
)
guardar_grafico(fig_eficiencia, 'eficiencia_comparada')

# 2.5 Tendencia Anual de Generación
fig_tendencia = px.bar(
    df_tendencia_anual,
    x='Año',
    y='Generacion_GWh',
    color='Turbina',
    color_discrete_sequence=paleta_secundaria,
    title='Generación Anual por Turbina (GWh)',
    labels={'Generacion_GWh': 'Generación (GWh)'},
    height=500
)
fig_tendencia.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='#f8f9fa',
    barmode='group'
)
guardar_grafico(fig_tendencia, 'tendencia_anual')

# 2.6 Comparación entre Turbinas
fig_comparacion = px.box(
    df_generacion,
    x='Turbina',
    y='Generacion_MWh',
    color='Turbina',
    color_discrete_sequence=paleta_secundaria,
    title='Distribución de Generación por Turbina (MWh)',
    labels={'Generacion_MWh': 'Generación (MWh)'},
    height=500
)
fig_comparacion.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='#f8f9fa',
    showlegend=False
)
guardar_grafico(fig_comparacion, 'comparacion_turbinas')

# 2.7 Rosa de los Vientos
fig_rosa = px.bar_polar(
    df_rosa_vientos,
    r='Frecuencia_Relativa',
    theta='Direccion',
    color='Velocidad_Promedio_m/s',
    color_continuous_scale=paleta_colores,
    title='Rosa de los Vientos - Frecuencia y Velocidad',
    height=600
)
fig_rosa.update_layout(
    polar=dict(
        radialaxis=dict(visible=True),
        angularaxis=dict(direction='clockwise')
    ),
    paper_bgcolor='#f8f9fa'
)
guardar_grafico(fig_rosa, 'rosa_vientos')

# 2.8 Curva de Potencia
fig_curva = go.Figure()

# Curva teórica
fig_curva.add_trace(go.Scatter(
    x=df_curva_potencia['Velocidad_Viento'],
    y=df_curva_potencia['Potencia_Teorica_kW'],
    name='Teórica',
    line=dict(color=paleta_colores[0], width=3, dash='dot')
))

# Curva real
fig_curva.add_trace(go.Scatter(
    x=df_curva_potencia['Velocidad_Viento'],
    y=df_curva_potencia['Potencia_Real_kW'],
    name='Real',
    line=dict(color=paleta_colores[2], width=4)
))

# Eficiencia
fig_curva.add_trace(go.Scatter(
    x=df_curva_potencia['Velocidad_Viento'],
    y=df_curva_potencia['Eficiencia_Relativa']*1500,
    name='Eficiencia',
    line=dict(color=paleta_colores[4], width=2),
    yaxis='y2'
))

fig_curva.update_layout(
    title='Curva de Potencia y Eficiencia',
    xaxis_title='Velocidad del Viento (m/s)',
    yaxis_title='Potencia (kW)',
    yaxis2=dict(
        title='Eficiencia (%)',
        overlaying='y',
        side='right',
        range=[0, 1500],
        tickvals=[0, 300, 600, 900, 1200, 1500],
        ticktext=['0%', '20%', '40%', '60%', '80%', '100%']
    ),
    plot_bgcolor='white',
    paper_bgcolor='#f8f9fa',
    height=600
)
guardar_grafico(fig_curva, 'curva_potencia')

# =================================================================
# 3. GENERAR ARCHIVO INDEX CON TODOS LOS GRÁFICOS
# =================================================================

html_content = f"""
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Dashboard Eólico - Visualizaciones</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #f8f9fa;
        }}
        .header {{
            background-color: #017c42;
            color: white;
            padding: 20px;
            text-align: center;
            border-radius: 8px;
            margin-bottom: 30px;
        }}
        .grid-container {{
            display: grid;
            grid-template-columns: repeat(2, 1fr);
            gap: 20px;
        }}
        .graph-container {{
            background-color: white;
            border-radius: 8px;
            padding: 15px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        .graph-title {{
            color: #017c42;
            margin-top: 0;
            font-size: 18px;
        }}
        iframe {{
            width: 100%;
            height: 500px;
            border: none;
        }}
        .section-title {{
            color: #209559;
            grid-column: 1 / -1;
            border-bottom: 2px solid #3eae6f;
            padding-bottom: 10px;
        }}
    </style>
</head>
<body>
    <div class="header">
        <h1>Dashboard de Energía Eólica</h1>
        <p>Visualizaciones generadas con Python y Plotly</p>
    </div>

    <div class="grid-container">
        <h2 class="section-title">Resumen</h2>

        <div class="graph-container">
            <h3 class="graph-title">Generación Mensual por Turbina</h3>
            <iframe src="generacion_mensual.html"></iframe>
        </div>

        <div class="graph-container">
            <h3 class="graph-title">Factor de Capacidad Mensual</h3>
            <iframe src="factor_capacidad.html"></iframe>
        </div>

        <div class="graph-container">
            <h3 class="graph-title">Disponibilidad de Turbinas</h3>
            <iframe src="disponibilidad_turbinas.html"></iframe>
        </div>

        <div class="graph-container">
            <h3 class="graph-title">Eficiencia Comparada</h3>
            <iframe src="eficiencia_comparada.html"></iframe>
        </div>

        <h2 class="section-title">Tendencias</h2>

        <div class="graph-container">
            <h3 class="graph-title">Tendencia Anual de Generación</h3>
            <iframe src="tendencia_anual.html"></iframe>
        </div>

        <div class="graph-container">
            <h3 class="graph-title">Comparación entre Turbinas</h3>
            <iframe src="comparacion_turbinas.html"></iframe>
        </div>

        <h2 class="section-title">Análisis del Viento</h2>

        <div class="graph-container" style="grid-column: span 2;">
            <h3 class="graph-title">Rosa de los Vientos</h3>
            <iframe src="rosa_vientos.html"></iframe>
        </div>

        <div class="graph-container" style="grid-column: span 2;">
            <h3 class="graph-title">Curva de Potencia</h3>
            <iframe src="curva_potencia.html"></iframe>
        </div>
    </div>
</body>
</html>
"""

with open('graficas_eolicas/index.html', 'w') as f:
    f.write(html_content)

print("\n¡Todos los gráficos han sido generados exitosamente!")
print(f"Puedes abrir el archivo 'graficas_eolicas/index.html' en tu navegador para ver el dashboard completo.")

Gráfico guardado: graficas_eolicas/generacion_mensual.html
Gráfico guardado: graficas_eolicas/factor_capacidad.html
Gráfico guardado: graficas_eolicas/disponibilidad_turbinas.html
Gráfico guardado: graficas_eolicas/eficiencia_comparada.html
Gráfico guardado: graficas_eolicas/tendencia_anual.html
Gráfico guardado: graficas_eolicas/comparacion_turbinas.html
Gráfico guardado: graficas_eolicas/rosa_vientos.html
Gráfico guardado: graficas_eolicas/curva_potencia.html

¡Todos los gráficos han sido generados exitosamente!
Puedes abrir el archivo 'graficas_eolicas/index.html' en tu navegador para ver el dashboard completo.
